In [10]:
from numpy.core.fromnumeric import mean
from flask import jsonify
import pandas as pd
import numpy as np
import json
import re
import http
import xgboost as xgb
import time
import urllib
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
from itertools import chain, starmap
%load_ext jupyternotify

# years completed 2002-2020
years_arr = range(2001,2005)
def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException, http.client.IncompleteRead) as e: 
        print('Download error:', url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
        if num_retries > 0: 
            if e == http.client.IncompleteRead: 
                time.sleep(10)
                return download(url, num_retries - 1)
    return html

def flatten_json_iterative(dictionary, sep = '.', ind_start = 0):
    """Flattening a nested json file"""

    def unpack_one(parent_key, parent_value):
        """Unpack one level (only one) of nesting in json file"""
        # Unpacking one level        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                t1 = parent_key + sep + key
                yield t1, value
        elif isinstance(parent_value, list):
            i = ind_start 
            for value in parent_value:
                t2 = parent_key + sep +str(i) 
                i += 1
                yield t2, value
        else:
            yield parent_key, parent_value    

            
    # Continue iterating the unpack_one function until the terminating condition is satisfied
    while True:
        # Continue unpacking the json file until all values are atomic elements (aka neither a dictionary nor a list)
        dictionary = dict(chain.from_iterable(starmap(unpack_one, dictionary.items())))
        # Terminating condition: none of the values in the json file are a dictionary or a list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

def nba_pbp(gameId):
        """nba_pbp()
        Pull the game by id
        Data from API endpoints:
        * nba/playbyplay
        * nba/summary
        """
        # play by play
        pbp_url = "http://cdn.espn.com/core/nba/playbyplay?gameId={}&xhr=1&render=false&userab=18".format(gameId)
        pbp_resp = download(url=pbp_url)
        pbp_txt = {}
        if pbp_resp is not None:
            pbp_txt['teams'] = np.array([])
            pbp_txt['id'] = np.array([])
            pbp_txt['competitions'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['header'] = {}
            pbp_txt['pickcenter'] = np.array([])
            pbp_txt['broadcasts'] = np.array([])
            pbp_txt['videos'] = np.array([])
            pbp_txt['standings'] = np.array([])
            pbp_txt['boxscore'] = np.array([])
            pbp_txt['espnWP'] = np.array([])
            pbp_txt['gameInfo'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['timeouts'] = {}
            pbp_d = json.loads(pbp_resp)
            pbp_txt = pbp_d['gamepackageJSON']
            pbp_txt['gameId'] = pbp_d['gameId']
            # summary endpoint for pickcenter array
            summary_url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/summary?event={}".format(gameId)
            summary_resp = download(url=summary_url)
            summary = json.loads(summary_resp)
            
            if 'pickcenter' in summary.keys():
                summary_txt = summary['pickcenter']
            else:
                summary_txt = {}
                summary_txt['pickcenter']={}
            # ESPN's win probability
            wp = "winprobability"
            if wp in summary:
                espnWP = summary["winprobability"]
            else:
                espnWP = np.array([])
            if 'news' in pbp_txt.keys():
                del pbp_txt['news']
            if 'shop' in pbp_txt.keys():
                del pbp_txt['shop']
            pbp_txt['gameInfo'] = pbp_txt['header']['competitions'][0]
            pbp_txt['season'] = pbp_txt['header']['season']['year']
            pbp_txt['pickcenter'] = summary_txt
            pbp_txt['espnWP'] = espnWP
            # Home and Away identification variables
            homeTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['id'])
            awayTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['id'])
            homeTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['name'])
            awayTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['name'])
            homeTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['location'])
            awayTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['location'])
            homeTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['abbreviation'])
            awayTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['abbreviation'])
            homeTeamNameAlt = re.sub("Stat(.+)", "St", str(homeTeamName))
            awayTeamNameAlt = re.sub("Stat(.+)", "St", str(awayTeamName))

            if len(pbp_txt['espnWP']) > 1:
                pbp_txt['espnWP'] = espnWP
            else:
                pbp_txt['espnWP'] = espnWP
            pbp_txt['plays_mod'] = []
            for play in pbp_txt['plays']:
                p = flatten_json_iterative(play)
                pbp_txt['plays_mod'].append(p)
            pbp_txt['plays'] = pd.json_normalize(pbp_txt,'plays_mod')
            if len(pbp_txt['plays'])>1:
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                # Spread definition
                pbp_txt['plays']["homeTeamSpread"] = 2.5
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                pbp_txt['plays'] = pd.DataFrame(pbp_txt['plays'])
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']['period.number'] = pbp_txt['plays']['period.number'].apply(lambda x: int(x))
                #----- Figuring out Timeouts ---------
                pbp_txt['timeouts'] = {}
                pbp_txt['timeouts'][homeTeamId] = {"1": [], "2": []}
                pbp_txt['timeouts'][awayTeamId] = {"1": [], "2": []}
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                #----- Time ---------------
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.displayValue'].str.split(pat=':')
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.mm'].apply(lambda m: ['0'] + m if len(m) < 2 else m)
                pbp_txt['plays'][['clock.minutes','clock.seconds']] = pbp_txt['plays']['clock.mm'].to_list()
                pbp_txt['plays']['half'] = np.where(pbp_txt['plays']['period.number'] <= 2, "1","2")
                pbp_txt['plays']['lag_half'] = pbp_txt['plays']['half'].shift(1)
                pbp_txt['plays']['lead_half'] = pbp_txt['plays']['half'].shift(-1)

                pbp_txt['plays']['game_play_number'] = np.arange(len(pbp_txt['plays']))+1
                pbp_txt['plays']['text'] = pbp_txt['plays']['text'].astype(str)
                pbp_txt['plays']['id'] = pbp_txt['plays']['id'].apply(lambda x: int(x))
                del pbp_txt['plays']['clock.mm']
            else:
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
            
            
            pbp_json = {
                "gameId": pbp_txt['gameId'],
                "plays" : pbp_txt['plays'].to_dict(orient='records'),
                "winprobability" : np.array(pbp_txt['winprobability']).tolist(),
                "boxscore" : pbp_txt['boxscore'],
                "header" : pbp_txt['header'],
                "homeTeamSpread" : np.array(pbp_txt['homeTeamSpread']).tolist(),
                "broadcasts" : np.array(pbp_txt['broadcasts']).tolist(),
                "videos" : np.array(pbp_txt['videos']).tolist(),
                "standings" : pbp_txt['standings'],
                "pickcenter" : np.array(pbp_txt['pickcenter']).tolist(),
                "espnWP" : np.array(pbp_txt['espnWP']).tolist(),
                "gameInfo" : np.array(pbp_txt['gameInfo']).tolist(),
                "season" : np.array(pbp_txt['season']).tolist()
            }
            return pbp_json, pbp_json['season']
        else:
            pass
        
    
schedule = pd.read_csv('nba_games_info_2002_2021.csv', encoding='latin-1')
schedule = schedule.sort_values(by=['season', 'date'], ascending = False)

games = schedule[(schedule['season'].isin(years_arr))&(schedule['status.type.name']=='STATUS_FINAL')].reset_index()['id']
print(f"Number of Games: {len(games)}, first gameId: {games[0]}")
g, y = nba_pbp(gameId = games[0])
# for key in g.keys():
#     print(f"{key}: {type(g[key])}")    
json.dumps(g,indent=4)
print(g.keys())
print(pd.DataFrame(g['plays']))
y

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


2021-02-28 07:55:06,570 [46056] WARNING  py.warnings:109: [JupyterRequire] C:\Users\saiem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (14,50,63,64,70,71,72) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Number of Games: 4146, first gameId: 240615008
dict_keys(['gameId', 'plays', 'winprobability', 'boxscore', 'header', 'homeTeamSpread', 'broadcasts', 'videos', 'standings', 'pickcenter', 'espnWP', 'gameInfo', 'season'])
     shootingPlay sequenceNumber period.displayValue  period.number  \
0           False              1         1st Quarter              1   
1           False              2         1st Quarter              1   
2           False              3         1st Quarter              1   
3           False              4         1st Quarter              1   
4            True              5         1st Quarter              1   
..            ...            ...                 ...            ...   
473         False            478         4th Quarter              4   
474          True            479         4th Quarter              4   
475         False            480         4th Quarter              4   
476         False            481         4th Quarter              4   


2004

In [11]:
%%notify
# 2020:
# 2019: 
# 2018:
# 2017:
# 2016:
# 2015:
# 2014:
# 2013:
# 2012:
# 2011:
# 2010:
# 2009:
# 2008:
# 2007:
# 2006: 
# 2005:
# 2004:
# 2003:
# 2002:
i = 0
for game in games[i:]:
    print(f"Working on game {i+1} of {len(games)}, gameId: {games[i]}")
    if len(str(game))<9:
        i+=1
        continue
    try:
        g, y = nba_pbp(gameId=game)
    except (TypeError) as e: 
        print("TypeError: yo", e)
        g, y = nba_pbp(gameId=game)
        
    except (KeyError) as e:
        print("KeyError: yo", e)
        i+=1
        continue
    except (ValueError) as e:
        print("DecodeError: yo", e)
        i+=1
        continue
    fp = "nba/{}/{}.json".format(y, game)
    with open(fp,'w') as f:
        json.dump(g, f, indent=4, sort_keys=False) 
#     time.sleep(1)        
    i+=1

Working on game 1 of 4146, gameId: 240615008
Working on game 2 of 4146, gameId: 240613008
Working on game 3 of 4146, gameId: 240610008
Working on game 4 of 4146, gameId: 240608013
Working on game 5 of 4146, gameId: 240606013
Working on game 6 of 4146, gameId: 240601008
Working on game 7 of 4146, gameId: 240531013
Working on game 8 of 4146, gameId: 240530011
Working on game 9 of 4146, gameId: 240529016
Working on game 10 of 4146, gameId: 240528008
Working on game 11 of 4146, gameId: 240527013
Working on game 12 of 4146, gameId: 240526008
Working on game 13 of 4146, gameId: 240525013
Working on game 14 of 4146, gameId: 240524011
Working on game 15 of 4146, gameId: 240523016
Working on game 16 of 4146, gameId: 240522011
Working on game 17 of 4146, gameId: 240521016
Working on game 18 of 4146, gameId: 240520008
Working on game 19 of 4146, gameId: 240519016
Working on game 20 of 4146, gameId: 240518014
Working on game 21 of 4146, gameId: 240516017
Working on game 22 of 4146, gameId: 2405160

Working on game 178 of 4146, gameId: 240402015
Working on game 179 of 4146, gameId: 240402025
Working on game 180 of 4146, gameId: 240402007
Working on game 181 of 4146, gameId: 240402016
Working on game 182 of 4146, gameId: 240402019
Working on game 183 of 4146, gameId: 240402026
Working on game 184 of 4146, gameId: 240402029
Working on game 185 of 4146, gameId: 240401013
Working on game 186 of 4146, gameId: 240401006
Working on game 187 of 4146, gameId: 240401020
Working on game 188 of 4146, gameId: 240331001
Working on game 189 of 4146, gameId: 240331008
Working on game 190 of 4146, gameId: 240331024
Working on game 191 of 4146, gameId: 240331028
Working on game 192 of 4146, gameId: 240331002
Working on game 193 of 4146, gameId: 240331011
Working on game 194 of 4146, gameId: 240331016
Working on game 195 of 4146, gameId: 240331019
Working on game 196 of 4146, gameId: 240331026
Working on game 197 of 4146, gameId: 240331027
Working on game 198 of 4146, gameId: 240330006
Working on ga

Working on game 353 of 4146, gameId: 240309023
Working on game 354 of 4146, gameId: 240309025
Working on game 355 of 4146, gameId: 240309027
Working on game 356 of 4146, gameId: 240309029
Working on game 357 of 4146, gameId: 240308001
Working on game 358 of 4146, gameId: 240308006
Working on game 359 of 4146, gameId: 240308020
Working on game 360 of 4146, gameId: 240308026
Working on game 361 of 4146, gameId: 240307007
Working on game 362 of 4146, gameId: 240307012
Working on game 363 of 4146, gameId: 240307016
Working on game 364 of 4146, gameId: 240307009
Working on game 365 of 4146, gameId: 240307019
Working on game 366 of 4146, gameId: 240307025
Working on game 367 of 4146, gameId: 240307027
Working on game 368 of 4146, gameId: 240307028
Working on game 369 of 4146, gameId: 240307013
Working on game 370 of 4146, gameId: 240307010
Working on game 371 of 4146, gameId: 240306005
Working on game 372 of 4146, gameId: 240306007
Working on game 373 of 4146, gameId: 240306020
Working on ga

Working on game 524 of 4146, gameId: 240211026
Working on game 525 of 4146, gameId: 240211029
Working on game 526 of 4146, gameId: 240210021
Working on game 527 of 4146, gameId: 240210004
Working on game 528 of 4146, gameId: 240210006
Working on game 529 of 4146, gameId: 240210014
Working on game 530 of 4146, gameId: 240210017
Working on game 531 of 4146, gameId: 240210015
Working on game 532 of 4146, gameId: 240210016
Working on game 533 of 4146, gameId: 240210022
Working on game 534 of 4146, gameId: 240210025
Working on game 535 of 4146, gameId: 240209010
Working on game 536 of 4146, gameId: 240209001
Working on game 537 of 4146, gameId: 240209005
Working on game 538 of 4146, gameId: 240209007
Working on game 539 of 4146, gameId: 240208017
Working on game 540 of 4146, gameId: 240208018
Working on game 541 of 4146, gameId: 240208009
Working on game 542 of 4146, gameId: 240208023
Working on game 543 of 4146, gameId: 240208011
Working on game 544 of 4146, gameId: 240208016
Working on ga

Working on game 699 of 4146, gameId: 240118002
Working on game 700 of 4146, gameId: 240117017
Working on game 701 of 4146, gameId: 240117004
Working on game 702 of 4146, gameId: 240117013
Working on game 703 of 4146, gameId: 240117015
Working on game 704 of 4146, gameId: 240117001
Working on game 705 of 4146, gameId: 240117003
Working on game 706 of 4146, gameId: 240117022
Working on game 707 of 4146, gameId: 240117026
Working on game 708 of 4146, gameId: 240117027
Working on game 709 of 4146, gameId: 240117010
Working on game 710 of 4146, gameId: 240117020
Working on game 711 of 4146, gameId: 240116008
Working on game 712 of 4146, gameId: 240116012
Working on game 713 of 4146, gameId: 240116018
Working on game 714 of 4146, gameId: 240116023
Working on game 715 of 4146, gameId: 240116007
Working on game 716 of 4146, gameId: 240116011
Working on game 717 of 4146, gameId: 240116019
Working on game 718 of 4146, gameId: 240116021
Working on game 719 of 4146, gameId: 240115009
Working on ga

...

Working on game 3581 of 4146, gameId: 220104002
Working on game 3582 of 4146, gameId: 220104013
Working on game 3583 of 4146, gameId: 220104017
Working on game 3584 of 4146, gameId: 220104024
Working on game 3585 of 4146, gameId: 220104007
Working on game 3586 of 4146, gameId: 220104016
Working on game 3587 of 4146, gameId: 220104025
Working on game 3588 of 4146, gameId: 220104027
Working on game 3589 of 4146, gameId: 220104028
Working on game 3590 of 4146, gameId: 220104029
Working on game 3591 of 4146, gameId: 220103010
Working on game 3592 of 4146, gameId: 220103018
Working on game 3593 of 4146, gameId: 220103003
Working on game 3594 of 4146, gameId: 220102004
Working on game 3595 of 4146, gameId: 220102017
Working on game 3596 of 4146, gameId: 220102024
Working on game 3597 of 4146, gameId: 220102002
Working on game 3598 of 4146, gameId: 220102005
Working on game 3599 of 4146, gameId: 220102007
Working on game 3600 of 4146, gameId: 220102016
Working on game 3601 of 4146, gameId: 22

Working on game 3752 of 4146, gameId: 211209028
Working on game 3753 of 4146, gameId: 211208004
Working on game 3754 of 4146, gameId: 211208005
Working on game 3755 of 4146, gameId: 211208006
Working on game 3756 of 4146, gameId: 211208015
Working on game 3757 of 4146, gameId: 211208018
Working on game 3758 of 4146, gameId: 211208001
Working on game 3759 of 4146, gameId: 211208016
Working on game 3760 of 4146, gameId: 211208022
Working on game 3761 of 4146, gameId: 211208026
Working on game 3762 of 4146, gameId: 211208029
Working on game 3763 of 4146, gameId: 211207002
Working on game 3764 of 4146, gameId: 211207009
Working on game 3765 of 4146, gameId: 211207012
Working on game 3766 of 4146, gameId: 211207017
Working on game 3767 of 4146, gameId: 211207019
Working on game 3768 of 4146, gameId: 211207024
Working on game 3769 of 4146, gameId: 211207011
Working on game 3770 of 4146, gameId: 211207023
Working on game 3771 of 4146, gameId: 211207025
Working on game 3772 of 4146, gameId: 21

Working on game 3923 of 4146, gameId: 211114009
Working on game 3924 of 4146, gameId: 211114012
Working on game 3925 of 4146, gameId: 211114002
Working on game 3926 of 4146, gameId: 211114005
Working on game 3927 of 4146, gameId: 211114020
Working on game 3928 of 4146, gameId: 211114027
Working on game 3929 of 4146, gameId: 211114021
Working on game 3930 of 4146, gameId: 211113014
Working on game 3931 of 4146, gameId: 211113018
Working on game 3932 of 4146, gameId: 211113024
Working on game 3933 of 4146, gameId: 211113007
Working on game 3934 of 4146, gameId: 211113011
Working on game 3935 of 4146, gameId: 211113016
Working on game 3936 of 4146, gameId: 211113025
Working on game 3937 of 4146, gameId: 211113023
Working on game 3938 of 4146, gameId: 211112026
Working on game 3939 of 4146, gameId: 211112029
Working on game 3940 of 4146, gameId: 211111007
Working on game 3941 of 4146, gameId: 211111013
Working on game 3942 of 4146, gameId: 211111008
Working on game 3943 of 4146, gameId: 21

Working on game 4094 of 4146, gameId: 211018029
Working on game 4095 of 4146, gameId: 211017004
Working on game 4096 of 4146, gameId: 211017019
Working on game 4097 of 4146, gameId: 211017005
Working on game 4098 of 4146, gameId: 211017011
Working on game 4099 of 4146, gameId: 211017016
Working on game 4100 of 4146, gameId: 211017021
Working on game 4101 of 4146, gameId: 211017022
Working on game 4102 of 4146, gameId: 211016006
Working on game 4103 of 4146, gameId: 211016008
Working on game 4104 of 4146, gameId: 211016012
Working on game 4105 of 4146, gameId: 211016013
Working on game 4106 of 4146, gameId: 211016024
Working on game 4107 of 4146, gameId: 211016003
Working on game 4108 of 4146, gameId: 211016026
Working on game 4109 of 4146, gameId: 211016028
Working on game 4110 of 4146, gameId: 211015015
Working on game 4111 of 4146, gameId: 211015022
Working on game 4112 of 4146, gameId: 211015029
Working on game 4113 of 4146, gameId: 211014001
Working on game 4114 of 4146, gameId: 21

<IPython.core.display.Javascript object>